In [ ]:
from pyhessian import hessian #import specific libraries, use pip install pyhessian

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from math import ceil, sqrt
#standard python imports

In neural network training we can view the loss space of a function in regards to the Sherrington-Kirkpatrick model, which gives a configuration of neurons (of **N** size and of variance **{-1,1}**) where the weights of each neuron configuration can be mapped as W (i,j), W describes the 'coupling weights' (connections between neurons). Viewing the Hamiltonian of this space allows the Hessian to be taken, since this space is in regards to model loss, the most negative space is seen as the most accurate neuron configuration.

In [ ]:
#This verision is depriciated because it was written with numpy and then converted

'''
def spatial_hessian_descent2(step_size, dimensionality, start_position, hessian_fn, n_steps=None):
  """
  This code uses the most negative eigenvector of the Hessian, since this value is 2v^T ∇^2Hₙ(σq)v is as negative as possible
  We do not use the Gradient of the hessian (first order derivative) because "we should have ∇HN (σq) ≈ 0, so we probably should not choose v
  based on the gradient." this is asserting using the first derivate of the Hessian will lead us to local minima and not the global minimum.

  """
  #takes in the step size N as a float value
  #the dimensionality of the data
  #the subset of

  if n_steps is None:
    #default if no number is given
    n_steps = ceil(1 / step_size)

  position = start_position.copy()
  trajectory = [position.copy()]

  for i in range(n_steps):
        # Get current Hessian
        hess = hessian_fn(position)

  for i in range[ceil(1/n)]:
    print('67')
    #top_eigenvalues, top_eigenvector = fn.eigenvalues() # top eigven value and top eigen vector
  return trajectory
  '''


'\ndef spatial_hessian_descent2(step_size, dimensionality, start_position, hessian_fn, n_steps=None):\n  """\n  This code uses the most negative eigenvector of the Hessian, since this value is 2v^T ∇^2Hₙ(σq)v is as negative as possible\n  We do not use the Gradient of the hessian (first order derivative) because "we should have ∇HN (σq) ≈ 0, so we probably should not choose v\n  based on the gradient." this is asserting using the first derivate of the Hessian will lead us to local minima and not the global minimum.\n\n  """\n  #takes in the step size N as a float value\n  #the dimensionality of the data\n  #the subset of\n\n  if n_steps is None:\n    #default if no number is given\n    n_steps = ceil(1 / step_size)\n\n  position = start_position.copy()\n  trajectory = [position.copy()]\n\n  for i in range(n_steps):\n        # Get current Hessian\n        hess = hessian_fn(position)\n\n  for i in range[ceil(1/n)]:\n    print(\'67\')\n    #top_eigenvalues, top_eigenvector = fn.eigenvalue

**NOTE**
---
Currently this is using NumPy as the basis for matrix multiplication abd calculating the hessians, eigenvalues/vectors, etc.

Switching to torch will allow this to run on CUDA and most likely in parallel, which I am still looking into. doing this will increase effeciency exponentially (if not more)

In [ ]:
import torch
def spatial_hessian_descent(step_size, dimensionality, start_position, hessian_fn, gradient_fn=None, n_steps=None):
    """
    This code uses the most negative eigenvector of the Hessian, since this value is 2v^T ∇^2Hₙ(σq)v is as negative as possible
    We do not use the Gradient of the hessian (first order derivative) because "we should have ∇HN (σq) ≈ 0, so we probably should not choose v
    based on the gradient." this is asserting using the first derivate of the Hessian will lead us to local minima and not the global minimum.

    Args:
        step_size: float, the step size (τ in the paper, typically 1/k),
        dimensionality: int, N dimension of the space
        start_position: array, starting point σ_0
        hessian_fn: function that returns the Hessian matrix at a point
        gradient_fn: optional, gradient function (used only to pick sign of direction)
        n_steps: int, number of steps k (default: ceil(1/step_size))

    Returns:
        trajectory: list of positions along the path
    """
    if n_steps is None:
        n_steps = ceil(1 / step_size)

    position = start_position.clone().to(device)
    N = dimensionality #dimensionality
    trajectory = [position.clone()]


    for i in range(n_steps):
        # Get current Hessian
        #hessian, the second derivative loss function
        hess = hessian_fn(position)

        #projects Hessian to orthogonal subspace of current position
        #this is needed because we can only move orthogonally to the sphere
        norm_sq = torch.dot(position, position)

        if norm_sq > 1e-12:
            M = torch.eye(N, device=device) - torch.outer(position, position) / norm_sq
            hess_proj = M @ hess @ M
            #projects to tangent space ensures that the only directions to move or orhtogonal
        else:
            hess_proj = hess

        # get eigenvalues and eigenvectors (sorted ascending, so index 0 is most negative)
        #hess_proj = torch.from_numpy(hess_proj)
        eigenvalues, eigenvectors = torch.linalg.eigh(hess_proj)


        #can also be replaced by PyHessian when in NN applications
        #top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues() <---
        #where       hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

        # Find most negative eigenvector orthogonal to position
        v = eigenvectors[:, 0].clone()

        if norm_sq > 1e-12 and torch.dot(v, J @ position) < 0:
            v = -v

        position = position + sqrt(N) * step_size * v

        target_q = (i + 1) / n_steps
        target_radius = sqrt(N * target_q)
        position = position / torch.linalg.norm(position) * target_radius
        #goes back from tangent space to the sphere
        print(position)
        trajectory.append(position.clone())
        #has to return current position

    return trajectory

**NOTE**
---
The current architecture for traversing calcuates the eigenvalues of the Hessian each itteration through (which grows the time complexity of the code). In the case where p=2 we need only calculate the hessian once by
**H(σ)=−N​1​i<j∑​Jij​σi​σj**​ this can be done outside of our loop and run one calculation vs N

In [ ]:
def spatial_hessian_descent_p2(step_size, dimensionality, start_position, hessian_fn, gradient_fn=None, n_steps=None):
    """
    Optimized for p=2 SK model where Hessian is constant.

    Uses the most negative eigenvector of the Hessian, since 2v^T ∇²Hₙ(σq)v
    should be as negative as possible.

    Args:
        step_size: float, the step size (τ in the paper, typically 1/k)
        dimensionality: int, N dimension of the space
        start_position: tensor, starting point σ_0
        hessian_fn: function that returns the Hessian matrix at a point
        gradient_fn: optional, gradient function (used only to pick sign of direction)
        n_steps: int, number of steps k (default: ceil(1/step_size))

    Returns:
        trajectory: list of positions along the path
    """
    if n_steps is None:
        n_steps = ceil(1 / step_size)

    position = start_position.clone().to(device)
    N = dimensionality
    trajectory = [position.clone()]

    # Compute Hessian eigendecomposition ONCE (constant for p=2)
    hess = hessian_fn(position)
    eigenvalues, eigenvectors = torch.linalg.eigh(hess)

    for i in range(n_steps):

        norm_sq = torch.dot(position, position)

        # Find most negative eigenvector orthogonal to position
        for j in range(len(eigenvalues)):
            v = eigenvectors[:, j].clone()

            # Skip if v is parallel to position
            if norm_sq > 1e-10 and abs(torch.dot(v, position)) / torch.sqrt(norm_sq) > 0.99:
                continue

            # Orthogonalize v to position
            if norm_sq > 1e-10:
                v = v - torch.dot(v, position) * position / norm_sq
                v = v / torch.linalg.norm(v)

            # Flip sign if gradient points same direction as v
            if gradient_fn is not None and torch.dot(v, gradient_fn(position)) > 0:
                v = -v

            break

        position = position + sqrt(N) * step_size * v

        target_q = (i + 1) / n_steps
        target_radius = sqrt(N * target_q)
        position = position / torch.linalg.norm(position) * target_radius

        trajectory.append(position.clone())

    return trajectory

In [ ]:
def project_to_sphere(position, N):
    return position / torch.linalg.norm(position) * sqrt(N)

In [ ]:
def energy(sigma):
    return -torch.dot(sigma, J @ sigma) / (2 * N)
    #claude recommended this to check the optimization algortithim's effectiveness in traversal

In [ ]:
def visualize_plot():


In [ ]:
'''The paper proves that this greedy strategy achieves energy:
```
E(q) = -∫₀^q √(ν''(t)) dt
'''

"The paper proves that this greedy strategy achieves energy:\n```\nE(q) = -∫₀^q √(ν''(t)) dt\n"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#last minute, claude looked at a bottleneck in my code for calculating the eigenvalues
#using np instead of something with GPU capabilities (torch in this case)

In [ ]:
#SK model style
N = 1100
torch.manual_seed(40)
J = torch.randn(N, N, N, device=device) #p = 2, σ_i & σ_j
#^ this value must be updated when looking at gradients above p = 2
J = (J + J.T) / sqrt(2*N)
J.fill_diagonal_(0)


def my_hessian(sigma):
    #can be replaced for other styles of models
    #derives hessian from the gradient
    return -J

def my_gradient(sigma):
    #can also be replaced I believe
    #this is specific to the S-K model, I believe
    return -J @ sigma

#def hypothetical_hessian: derived from another model type

start = torch.zeros(N, device=device)
start[0] = 1e-8

trajectory = spatial_hessian_descent(
    step_size=0.01,
    dimensionality=N,
    start_position=start,
    hessian_fn=my_hessian,
    gradient_fn=my_gradient
)

print(f"Steps taken: {len(trajectory)}")
print(f"Final ||σ||²/N: {torch.linalg.norm(trajectory[-1])**2 / N:.4f}")
#resultant walk distance from origin expressed as a number, should be 1.0 to denote a full traversal
print(f"Final energy: {energy(trajectory[-1]):.4f}")
#should fall around the parisi optimal value energy of about - sqrt(2)

tensor([ 0.0160, -0.0955,  0.0743,  ..., -0.0173,  0.1097, -0.0247],
       device='cuda:0')
tensor([ 0.0225, -0.1460,  0.1281,  ..., -0.0600,  0.1433, -0.0186],
       device='cuda:0')
tensor([ 0.0277, -0.1695,  0.1371,  ..., -0.0427,  0.1860, -0.0370],
       device='cuda:0')
tensor([ 0.0321, -0.1862,  0.1390,  ..., -0.0201,  0.2238, -0.0559],
       device='cuda:0')
tensor([ 0.0358, -0.2174,  0.1741,  ..., -0.0508,  0.2415, -0.0497],
       device='cuda:0')
tensor([ 0.0393, -0.2292,  0.1725,  ..., -0.0279,  0.2732, -0.0670],
       device='cuda:0')
tensor([ 0.0424, -0.2565,  0.2043,  ..., -0.0575,  0.2866, -0.0600],
       device='cuda:0')
tensor([ 0.0453, -0.2653,  0.2006,  ..., -0.0344,  0.3148, -0.0764],
       device='cuda:0')
tensor([ 0.0480, -0.2902,  0.2305,  ..., -0.0634,  0.3255, -0.0689],
       device='cuda:0')
tensor([ 0.0507, -0.2972,  0.2254,  ..., -0.0401,  0.3514, -0.0847],
       device='cuda:0')
tensor([ 0.0531, -0.3204,  0.2539,  ..., -0.0686,  0.3604, -0.0768],
 

In [ ]:
print(f"||σ||² = {torch.dot(trajectory[-1], trajectory[-1]):.2f}")
print(f"N = {N}")
print(f"σᵀJσ = {torch.dot(trajectory[-1], J @ trajectory[-1]):.2f}")

||σ||² = 1100.00
N = 1100
σᵀJσ = 2192.90
